# 1. Introdução

As operações morfológicas são fundamentais no processamento de imagens binárias. Elas manipulam a estrutura dos objetos com base em um elemento estruturante, permitindo ajustar a forma dos objetos e limpar ruídos da imagem.

As duas operações básicas são:
- **Dilatação**: Expande regiões brancas (1s), preenchendo buracos e conectando partes desconectadas.
- **Erosão**: Encolhe regiões brancas, removendo ruídos e bordas finas.

Essas operações são definidas matematicamente por meio de conjuntos e transformações de translação e reflexão.

As operações morfológicas são definidas no contexto da Teoria dos Conjuntos, considerando a imagem como um conjunto de pixels com valor 1 (branco) e o **elemento estruturante** como um conjunto de deslocamentos aplicados sobre a imagem.

É uma pequena matriz binária (tipicamente 3x3 ou 5x5) usada como molde para determinar como a vizinhança de cada pixel será tratada.

In [ ]:
import numpy as np
import cv2
from pathlib import Path
from google.colab.patches import cv2_imshow  # Caso use Colab

kernel = np.array([[0,1,0],
                   [1,1,1],
                   [0,1,0]], dtype=np.uint8)
kernel

In [ ]:
import matplotlib.pyplot as plt
import math

def show_image(img, title):
  plt.figure(figsize=(10,5))
  plt.imshow(img)
  plt.title(title)
  plt.axis('off')
  plt.show()

### Erosão (⊖)

A **erosão** de uma imagem binária $A$ por um elemento estruturante $B$ é dada por:

$$
A \ominus B = \{ z \mid B_z \subseteq A \}
$$

Intuitivamente, o pixel central só permanece 1 se **todos os pixels** do kernel se ajustarem dentro da região branca da imagem. É usada para **remover ruído** ou **afinar bordas**.

### Dilatação (⊕)

A **dilatação** de uma imagem binária $A$ por $B$ é dada por:

$$
A \oplus B = \{ z \mid (\hat{B})_z \cap A \neq \emptyset \}
$$

Ou seja, se **qualquer pixel** do kernel sobreposto for 1 na imagem, o pixel central se torna 1. É usada para **preencher buracos** e **conectar componentes**.

In [ ]:
# Verifica se já foram baixadas as imagens do drive, baixando-as e descompactando se necessário
! [ ! -d f"/content/lena.png" ] && gdown -O /content/lena.png "16HLcddcqiAv92JsuJ0dbg9NKSz4D2dvj"

# Verifica se já foram baixadas as imagens do drive, baixando-as e descompactando se necessário
! [ ! -d f"/content/MAB.jpg" ] && gdown -O /content/MAB.jpg "1nb_YBLwcQVSXlliDnoByu27Oh2fzAeK8"


# 2. Exemplos

In [ ]:
def erode_manual(img, kernel):
    h, w = img.shape
    kh, kw = kernel.shape
    pad_h, pad_w = kh // 2, kw // 2
    padded = np.pad(img, ((pad_h, pad_h), (pad_w, pad_w)), mode='constant', constant_values=0)
    result = np.zeros_like(img)

    for i in range(h):
        for j in range(w):
            region = padded[i:i+kh, j:j+kw]
            if np.all(region[kernel==1] == 1):
                result[i, j] = 1
    return result

def dilate_manual(img, kernel):
    h, w = img.shape
    kh, kw = kernel.shape
    pad_h, pad_w = kh // 2, kw // 2
    padded = np.pad(img, ((pad_h, pad_h), (pad_w, pad_w)), mode='constant', constant_values=0)
    result = np.zeros_like(img)

    for i in range(h):
        for j in range(w):
            region = padded[i:i+kh, j:j+kw]
            if np.any(region[kernel==1] == 1):
                result[i, j] = 1
    return result

In [ ]:
import matplotlib.pyplot as plt

# Exemplo simples para teste
img = np.zeros((7,7), dtype=np.uint8)
img[2:5,2:5] = 1

kernel = np.ones((3,3), dtype=np.uint8)

# Aplicar operações
dil = dilate_manual(img, kernel)
ero = erode_manual(img, kernel)

# Visualizar
plt.figure(figsize=(12,3))
titles = ["Original", "Dilatação", "Erosão"]
for i, im in enumerate([img, dil, ero]):
    plt.subplot(1, 3, i+1)
    plt.imshow(im, cmap='gray')
    plt.title(titles[i])
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Criar imagem binária a partir de uma imagem real
img_cv = cv2.imread('lena.png', cv2.IMREAD_GRAYSCALE)
_, img_bin = cv2.threshold(img_cv, 128, 255, cv2.THRESH_BINARY)

kernel = np.ones((5,5), np.uint8)

# Aplicar operações morfológicas
erosion = cv2.erode(img_bin, kernel, iterations=1)
dilation = cv2.dilate(img_bin, kernel, iterations=1)

# Visualizar resultados
plt.figure(figsize=(12,4))
titles = ["Original Binária", "Dilatação", "Erosão"]
for i, im in enumerate([img_bin, dilation, erosion]):
    plt.subplot(1, 3, i+1)
    plt.imshow(im, cmap='gray')
    plt.title(titles[i])
    plt.axis('off')
plt.tight_layout()
plt.show()

# 3. Eliminação de Outliers

In [ ]:
img = cv2.imread("MAB.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

print(img.shape)
h, w, _ = img.shape

show_image(img, "Museu Aeroespacial Brasileiro")

In [ ]:
img_original = img.copy()

#Geração de máscara binária
gray_img = cv2.cvtColor(img_original, cv2.COLOR_RGB2GRAY)
binary_img = 255 * (gray_img > 200).astype(np.uint8)
cv2_imshow(binary_img)

In [ ]:
#criar duas versões de mascara com erosao ou dilatacao e visualizar

print("Errosão")
cv2_imshow(erosion)
print("Dilatação")
cv2_imshow(dilation)

## 4. Conclusão

- **Erosão** é usada para:
  - Remover ruído (pixels brancos isolados)
  - Afinar bordas

- **Dilatação** é usada para:
  - Preencher buracos
  - Conectar componentes próximos
